In [1]:
import pandas as pd
from typing import Optional, List
from core import system_msg, user_msg, assistant_msg, generic_call_stream
from qdrant_search import QdrantSearchEngine

In [2]:
engine = QdrantSearchEngine()

In [47]:
query = 'what is going on withmarketing'

In [48]:
query, instructions = query.split(':') if ':' in query else (query, '')
# Search for context using both methods with verbose mode for debugging
main_results = await engine.search(
    query_text=query,
    limit=200,
    min_score=0.2,
    exact_match_boost=2,  # Add boost parameter
    verbose=False  # Can be set to True for debugging
)


In [49]:
pd.options.display.max_colwidth = 1000

In [50]:
pd.DataFrame(main_results)


,score,topic_name,summary,details,meeting_id,speaker_name,topic_type,timestamp,vector_scores,exact_matches
0,0.262468,PolyMarket,PolyMarket is a prediction market platform that allows users to trade on the outcomes of future events.,"Dmitrii Bashkirov worked at PolyMarket, where he gained experience in marketing and product management.",23c077e5-a0f9-4175-8b5a-dd13bfb90c91,Dmitrii Bashkirov,company,2024-08-28T13:18:30.600000,"{'topic_name': 0.26246845454545453, 'summary': 0.11911140499999999}",[]
1,0.238831,Market competition and product differentiation,Dmitry and Özay discussed the crowded market for their products and the strategies needed to differentiate and succeed.,Dmitry acknowledged the presence of competition but argued that the market is still sparse. He emphasized the importance of understanding user needs and iterating on the product based on feedback. Özay raised concerns about the challenges of entering a crowded market and the need for significant investment to compete effectively.,50d5d659-3a57-42f2-8f32-31c7145ebe91,Dmitry Grankin,discussion,2024-10-24T09:32:32.980000,"{'topic_name': 0.2388306636363636, 'summary': 0.13504742045454546, 'details': 0.1130518318181818}",[]
2,0.235306,Marketing Challenges,"The team is facing difficulties in marketing their applications, leading to a decline in user engagement.","Despite having several tens of users, the marketing efforts are not yielding satisfactory results. The team is exploring paid traffic options after previously relying on organic growth through influencers. They are seeking advice from a Google manager to optimize their marketing budget.",82b19a91-cd0b-4add-a118-55eb4840e201,Dmitry Grankin,concern,2024-09-24T10:30:54.970000,"{'topic_name': 0.23530559090909087, 'summary': 0.11434869727272726, 'details': 0.0713872840909091}",[]
3,0.234946,product-market fit,Product-market fit is a critical concept discussed in relation to validating product ideas.,Dmitrii emphasizes the importance of understanding customer needs and feedback for achieving product-market fit.,dbf8ad27-649c-4a16-b5a1-5ab3db002bd9,Dmitrii Chistov,concept,2024-09-13T15:15:37.764000,"{'topic_name': 0.23494606818181818, 'summary': 0.12140329090909087, 'details': 0.08310836818181817}",[]
4,0.231642,marketing,Marketing involves promoting a product to attract users and generate interest during its launch.,"Effective marketing strategies are crucial for a successful launch on Product Hunt, including preparing engaging content and leveraging social media.",f8fd1366-6e33-4161-9ebe-d1aabdc2f433,Julia Zakharova,concept,2024-09-13T12:30:03.290000,"{'topic_name': 0.23164231818181819, 'summary': 0.13340694818181817, 'details': 0.07941820454545453}",[]
5,0.231642,Marketing,Marketing involves promoting products or services to potential customers.,Dmitry expresses his need to learn more about marketing as he is currently responsible for it in his company.,3540f190-1f83-4100-8d6f-e648752f336d,Dmitry Grankin,concept,2024-09-26T12:02:28.836000,"{'topic_name': 0.23164231818181819, 'summary': 0.12087514090909089, 'details': 0.07809450681818181}",[]
6,0.231642,marketing,"Marketing strategies are a central theme of the meeting, focusing on outreach and customer acquisition.",Dmitry emphasizes the need for effective marketing tactics to stand out in a competitive landscape.,dbf8ad27-649c-4a16-b5a1-5ab3db002bd9,Dmitry Grankin,concept,2024-09-13T15:15:37.764000,"{'topic_name': 0.23164231818181819, 'summary': 0.10366022227272727, 'details': 0.0928118409090909}",[]
7,0.222183,Market Research Insights,Olga compiled insights from Reddit and Product Hunt regarding the pros and cons of different solutions.,"She gathered relevant information and shared it in Notion, which will help in creating informed posts about competitors.",3ead8b34-f5de-42ef-8d28-5dd64a2b8669,Olga Nemirovskaya,update,2024-09-16T09:47:34.620000,{'topic_name': 0.22218325},[]
8,0.219941,Market Research,The team plans to conduct market research to better underst

In [ ]:

speaker_results = await engine.search_by_speaker(
    speaker_query=query,
    limit=200,
    min_score=0.4
)

In [31]:
speaker_results

[]

In [ ]:

class ChatSession:
    def __init__(self, search_engine: QdrantSearchEngine, prompts):
        self.search_engine = search_engine
        self.prompts = prompts
        self.messages: List = []
        self.results = None
    
    def add_message(self, role: str, content: str):
        """Add a message to the conversation history"""
        if role == "system":
            self.messages.append(system_msg(content))
        elif role == "user":
            self.messages.append(user_msg(content))
        elif role == "assistant":
            self.messages.append(assistant_msg(content))
    
    async def process_user_message(self, query: str):
        """Process a user message and generate a response"""
        self.add_message("user", query)
        
        # Split query and instructions if present
        query, instructions = query.split(':') if ':' in query else (query, '')
        # Search for context using both methods with verbose mode for debugging
        main_results = await self.search_engine.search(
            query_text=query,
            limit=200,
            min_score=0.4,
            exact_match_boost=0.3,  # Add boost parameter
            verbose=False  # Can be set to True for debugging
        )
        
        speaker_results = await self.search_engine.search_by_speaker(
            speaker_query=query,
            limit=200,
            min_score=0.49
        )
        
        # Convert results to DataFrames with new score fields
        main_df = pd.DataFrame(main_results) if main_results else pd.DataFrame()
        speaker_df = pd.DataFrame(speaker_results) if speaker_results else pd.DataFrame()
        
        # Select relevant columns and combine results
        columns = ['topic_name', 'speaker_name', 'summary', 'details']
        score_columns = ['score', 'vector_scores', 'exact_matches']  # Add new score columns
        
        if len(main_df) > 0:
            main_df = main_df[columns + score_columns]  # Include score information
            main_df['source'] = 'main'
        else:
            main_df = pd.DataFrame(columns=columns + score_columns + ['source'])
        
        if len(speaker_df) > 0:
            speaker_df = speaker_df[columns + ['score']]  # Speaker search has simpler scoring
            speaker_df['source'] = 'speaker'
        else:
            speaker_df = pd.DataFrame(columns=columns + ['score', 'source'])
        
        # Combine and deduplicate results
        self.results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)
        
        # Sort by score
        if not self.results.empty:
            self.results = self.results.sort_values('score', ascending=False)
        # Search for context using both methods
        main_results = await self.search_engine.search(
            query_text=query,
            limit=200,
            min_score=0.4
        )
        
        speaker_results = await self.search_engine.search_by_speaker(
            speaker_query=query,
            limit=200,
            min_score=0.49
        )
        
        # Convert results to DataFrames
        main_df = pd.DataFrame(main_results) if main_results else pd.DataFrame()
        
        speaker_df = pd.DataFrame(speaker_results) if speaker_results else pd.DataFrame()
        print(main_df.columns)
        print(speaker_df.columns)
        # Select relevant columns and combine results
        columns = ['topic_name', 'speaker_name', 'summary', 'details']
        main_df = main_df[columns] if len(main_df) > 0 else pd.DataFrame(columns=columns)
        main_df['source'] = 'main'
        speaker_df = speaker_df[columns] if len(speaker_df) > 0 else pd.DataFrame(columns=columns)
        speaker_df['source'] = 'speaker'
        
        # Combine and deduplicate results
        self.results = pd.concat([main_df, speaker_df]).drop_duplicates().reset_index(drop=True)
        
        # Convert results to markdown format for context
        context = self.results[columns].to_markdown(index=False) if not self.results.empty else "No relevant context found."
        
        # Add context message
        context_msg = system_msg(f"Context: {context}")
        
        # Generate response
        messages_with_context = [
            system_msg(self.prompts.perplexity),
            system_msg('special instructions: ' + instructions),
            *self.messages,
            context_msg
        ]
        
        response = await generic_call_stream(messages_with_context)
        
        # Add assistant response
        self.add_message("assistant", response)
        
        return response

    def get_results(self) -> Optional[pd.DataFrame]:
        """Get the latest search results"""
        return self.results

In [13]:
# Initialize
from prompts import Prompts
prompts = Prompts()
search_engine = QdrantSearchEngine()
chat = ChatSession(search_engine, prompts)

In [20]:

# Process a message
response = await chat.process_user_message("vexa")

/tmp/ipykernel_798206/3185951060.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)


Index(['score', 'topic_name', 'summary', 'details', 'meeting_id',
       'speaker_name', 'topic_type', 'timestamp', 'vector_scores',
       'exact_matches'],
      dtype='object')
RangeIndex(start=0, stop=0, step=1)
Vexa is a startup focused on developing a real-time meeting assistant that enhances communication and productivity during calls. The product is designed to transcribe conversations, summarize discussions, and provide contextual support, making it particularly useful for meetings conducted via platforms like Google Meet. Currently, Vexa is in the pre-seed stage and has been incorporated in the USA, with a developer version of the product available for testing and refinement[1][3][4][10].

The team is actively working on marketing strategies, including influencer partnerships and social media campaigns, to improve user acquisition and engagement, particularly in the Brazilian market[1][2][5]. Feedback from users has been generally positive, highlighting Vexa's clean interface

In [21]:
from IPython.display import Markdown
Markdown(response)

Vexa is a startup focused on developing a real-time meeting assistant that enhances communication and productivity during calls. The product is designed to transcribe conversations, summarize discussions, and provide contextual support, making it particularly useful for meetings conducted via platforms like Google Meet. Currently, Vexa is in the pre-seed stage and has been incorporated in the USA, with a developer version of the product available for testing and refinement[1][3][4][10].

The team is actively working on marketing strategies, including influencer partnerships and social media campaigns, to improve user acquisition and engagement, particularly in the Brazilian market[1][2][5]. Feedback from users has been generally positive, highlighting Vexa's clean interface and effective transcription capabilities, although some usability issues have been noted[4][8][9]. 

Vexa aims to differentiate itself from competitors by offering real-time transcription without the need for bots to join calls, thereby enhancing user experience and privacy[1][6][10]. The company is also exploring potential collaborations to leverage data services for better marketing insights and user growth[1][10].

In [22]:

# Get results
results_df = chat.get_results()
results_df

,topic_name,speaker_name,summary,details,source
0,Vexa,Dmitriy Grankin,Vexa is the product being marketed through inf...,The meeting discusses strategies for promoting...,main
1,Vexa,Olga Nemirovskaya,Vexa is a product being discussed in the meeti...,The team is considering how to improve user ac...,main
2,Vexa,Dmitry Grankin,"Vexa is a product discussed in the meeting, fo...",Vexa is currently in the testing phase with a ...,main
3,Vexa,Robert Hangu,Vexa is a real-time assistant for meetings tha...,Robert Hangu provides positive feedback about ...,main
4,Vexa,Dmitriy Grankin,Vexa is a venture-backed startup focused on de...,Vexa aims to provide tools that highlight key ...,main
...,...,...,...,...,...
70,Pricing strategy and potential lifetime deal,Umar Lateef,The discussion centered around VEXA's single p...,"Currently, VEXA has not launched billing or pa...",main
71,LinkedIn,Dmitry Grankin,LinkedIn is a social media platform used for m...,Dmitry states that Vexa has been posting on Li...,main
72,Dmitriy Grankin,Dmitriy Grankin,Dmitriy Grankin is a participant in the meetin...,Dmitriy explains how Vexa's real-time transcri...,main
73,real-time context,Dmitry Grankin,Real-time context refers to the ability of Vex...,Dmitry explains how Vexa collects and delivers...,main
